In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib
import bs4
import psycopg2
import sys
sys.path.append('/Users/kshain/Documents/Git')
from progressbar import ProgressBar
import sqlalchemy
import pandas.io.sql as psql
import time
import re
import datetime
%matplotlib inline  

# Play-by-play Data Scrape

## Table of contents
* [Connecting to PostgreSQL](#Connecting-to-PostgreSQL)
* [Scraping Game Data](#Scraping-Game-Data)
    * [Adding a season column](#Adding-a-season-column)
* [Scraping Play-by-play Data](#Scraping-Play-by-play-Data)
    * [Adding Year-to-date Percentage](#Adding-Year-to-date-Percentage)
    * [Adding Score ifferential](#Adding-Score-Differential)
* [Scraping Season Totals](#Scraping-Season-Totals)
    * [Adding Career-to-date](#Adding-Career-to-date)
    * [Casting to Proper Data Types](#Casting-to-Proper-Data-Types)

## Connecting to PostgreSQL

First, I am going to create a PostgreSQL database to store the data as it is scraped and processed.

I made the actual database by simply executing `createdb freethrows` in the bash terminal. To make the connection, I will use both `psycopg2` and `sqlalchemy` since each package has some useful functionality.

In [ ]:
conn = None

try:
    conn_string = "host='localhost',database='freethrows', user='kshain'"
    print('Connecting to database\n ->',conn_string)
    conn = psycopg2.connect(host='localhost',database='freethrows', user='kshain') 
    cur = conn.cursor()
    print('Connected!\n')            

except psycopg2.DatabaseError:
    print ('Error')  
    sys.exit(1)

In [ ]:
engine = sqlalchemy.create_engine('postgresql://kshain@localhost:5432/freethrows')

## Scraping Game Data

I'll start by going through all 66 seasons for which there is game data and scrape the data into the table `games`. I'm using the `BeautifulSoup` package which nicely organizes html for scraping. All of the `if` statements just make sure the correct field goes to the correct list. I make the list into a Pandas dataframe because `sqlalchemy` makes it really easy to go from dataframe to SQL table.

In [ ]:
pbar = ProgressBar.ProgressBar(66) 
for year in range(1950,2015+1):
    urlstring = 'http://www.basketball-reference.com/leagues/NBA_'+str(year)+'_games.html'
    source = urllib.request.urlopen(urlstring)
    source = source.read()
    soup = bs4.BeautifulSoup(source, 'lxml')
    tds = soup.find_all('td',attrs={'align':True})
    # initialize lists
    date = []
    gameid = []
    starttime = []
    teamv = []
    scorev = []
    teamh = []
    scoreh = []
    ot = []
    
    # scrape one year
    for i,row in enumerate(tds):
        if i%9==0:
            date.append(time.strftime('%Y-%m-%d',time.strptime(str(row.find('a').contents[0]), '%a, %b %d, %Y')))
            gameid.append(row['csk'])
        if i%9==1:
            try:
                starttime.append(row.contents[0])
            except:
                starttime.append('')
        if i%9==3:
            teamv.append(row.find('a').contents[0])
        if i%9==4:
            scorev.append(int(row.contents[0]))
        if i%9==5:
            teamh.append(row.find('a').contents[0])
        if i%9==6:
            scoreh.append(int(row.contents[0]))
        if i%9==7:
            ot.append(row.text)
    data = {'date':date,'gameid':gameid,'starttime':starttime,'teamv':teamv,'scorev':scorev,'teamh':teamh,'scoreh':scoreh,'ot':ot}
    
    #construct dataframe for games from one year
    gamesOneYear = pd.DataFrame(data,columns=['date','gameid','starttime','teamv','scorev','teamh','scoreh','ot'])
    
    gamesOneYear.to_sql('games',engine,if_exists='append')
    pbar.increment()
pbar.finish()

Because I will use it in scraping play-by-play data, I want to immediately cast the date into a `datetime` type variable.

In [ ]:
cur.execute("ALTER TABLE games ALTER COLUMN date TYPE DATE using to_date(date, 'YYYY-MM-DD');")
conn.commit()

### Adding a season column

Instead of extracting the year from the date frequently, I will encode that data separately in the `season` column.

In [ ]:
cur.execute("SELECT CAST(CASE WHEN EXTRACT(month FROM date)>7 THEN (EXTRACT(year FROM date)+1) ELSE (EXTRACT(year FROM Date)) END AS integer) FROM games")
seasondf = pd.DataFrame(cur.fetchall(), columns=['season'])
allgamesdf = psql.read_sql("SELECT * FROM games", conn)
allgamesdf = allgamesdf.join(seasondf)
allgamesdf.to_sql('allgames',engine,if_exists='append')

## Scraping Play-by-play Data

This contains the brunt of the scraping necessary for the more sophisticated models in the `PlayByPlayFreeThrow_DataScrape` notebook. I use the `gameid`'s from the last section to iterate through all the games with play-by-play data (2001 to present). Be aware that execution of this code will take a few hours.

In [ ]:
cur.execute("SELECT gameid FROM games WHERE EXTRACT(YEAR FROM date)>=2001")
gameids = cur.fetchall()
print('running')
pbar2 = ProgressBar.ProgressBar(len(gameids), finestep=True)

for j,thisgameid in enumerate(gameids):
    
    # Data to collect for each free throw
    gameid = []
    gametime = []
    player = []
    playerid = []
    team = []
    teamha = []
    opponent = []
    scorea = []
    scoreh = []
    result = []
    ftnumber = []
    tech = []
    
    urlstring = 'http://www.basketball-reference.com/boxscores/pbp/'+thisgameid[0]+'.html'

    source = urllib.request.urlopen(urlstring)
    source = source.read()
    soup = bs4.BeautifulSoup(source, 'lxml')
    pbptable = soup.find('table',attrs={'class':'no_highlight stats_table'})
    tdswFT = pbptable.findAll(['td','th'])

    ftrows = [t.parent for t in tdswFT if ('free throw' in t.text or (t.has_attr('colspan') and t['colspan']=='6'))]

    teama = pbptable.find('tr',attrs={'id':False}).contents[3].contents[0]
    teamh = pbptable.find('tr',attrs={'id':False}).contents[7].contents[0]
    
    period = 0 #initialize which quarter is being played

    for i,row in enumerate(ftrows):
        if row.has_attr('id'):
            period = period + 1
        else:
            gameid.append(thisgameid[0])
            timeinperiod = row.contents[1].contents[0]
            timeinperiod = float(timeinperiod.split(':')[0])+float(timeinperiod.split(':')[1])/60
            if period<=4:
                totaltime = 12.0*period-timeinperiod
            else:
                totaltime = 48.0+5.0*(period-4)-timeinperiod
            gametime.append(round(totaltime,3))
            home = row.contents[3].contents[0]=='\xa0'
            if home:
                player.append(row.contents[7].find('a').contents[0])
                playerid.append(row.contents[7].find('a')['href'].split('/')[-1].split('.')[0])
                team.append(teamh)
                opponent.append(teama)
                teamha.append('H')
                result.append(int('makes' in row.contents[7].text))
                try:
                    ftnumber.append(int(re.findall('\d+', row.contents[7].text)[0]))
                except:
                    ftnumber.append(1) 
                if 'technical' in row.contents[7].text:
                    tech.append(1)
                else:
                    tech.append(0)

            else:
                player.append(row.contents[3].find('a').contents[0])
                playerid.append(row.contents[3].find('a')['href'].split('/')[-1].split('.')[0])
                team.append(teama)
                opponent.append(teamh)
                teamha.append('A')
                result.append(int('makes' in row.contents[3].text))
                try:
                    ftnumber.append(int(re.findall('\d+', row.contents[3].text)[0]))
                except:
                    ftnumber.append(1)  
                if 'technical' in row.contents[3].text:
                    tech.append(1)
                else:
                    tech.append(0)
                    
            scores = row.contents[5].contents[0].split('-')
            scorea.append(scores[0])
            scoreh.append(scores[1])

    data = {'gameid':gameid,'gametime':gametime,'player':player,'playerid':playerid,'team':team,'teamha':teamha,'opponent':opponent,'scorea':scorea,'scoreh':scoreh,'result':result,'ftnumber':ftnumber,'tech':tech}

    oneGame = pd.DataFrame(data,columns=['gameid','gametime','player','playerid','team','teamha','opponent','scorea','scoreh','result','ftnumber','tech'])
    oneGame.to_sql('pbpfts',engine,if_exists='append')

    pbar2.increment()
pbar2.finish()

### Adding Year-to-date Percentage

It is important that we don't violate the flow of time so we should only be allowed to know the year-to-date free throw average of each player in order to use it as a parameter to our model.

In [ ]:
query = "SELECT * FROM pbpfts"
oldpbpfts = psql.read_sql(query, conn)

In [ ]:
query = "SELECT playerid,result,season FROM pbpfts JOIN allgames on pbpfts.gameid=allgames.gameid"
ytd_avg_needs = psql.read_sql(query, conn)

In [ ]:
ytd_avg = np.empty((2,ytd_avg_needs.shape[0]))

for i in range(ytd_avg_needs.shape[0]):
    ytd_avg_needs_temp = ytd_avg_needs.ix[max(0,i-100000):i]
    rowsToCount = (ytd_avg_needs_temp.season==ytd_avg_needs_temp.ix[i].season).values & (ytd_avg_needs_temp.playerid==ytd_avg_needs_temp.ix[i].playerid).values
    ytd_avg[0,i]= rowsToCount.sum()
    ytd_avg[1,i]=(ytd_avg_needs_temp[rowsToCount].result).sum()

ytd_additional_columns = pd.DataFrame(ytd_avg.transpose(), columns=['attempts_to_date','makes_to_date'])

In [ ]:
newpbpfts = oldpbpfts.join(ytd_additional_columns)
newpbpfts['ytd_ftpct'] = newpbpfts.makes_to_date/newpbpfts.attempts_to_date

### Adding Score Differential

The score differential is always calculated as the free throw shooting team minus the opposing team.

In [ ]:
newpbpfts['score_diff'] = (newpbpfts.scorea.astype(int)-newpbpfts.scoreh.astype(int))*(((newpbpfts.teamha=='A').values)*2-1)

Now we can go back to SQL.

In [ ]:
newpbpfts.to_sql('allpbpfts',engine,if_exists='append')

## Scraping Season Totals

The last category of useful data from basketball-reference.com is player statistics by season.

In [ ]:
yearRange = range(1950,2016)
pbar2 = ProgressBar.ProgressBar(len(yearRange), finestep=True)

for j,season in enumerate(yearRange):
    year = []
    player = []
    playerid = []
    team = []
    pos = []
    age = []
    ft = []
    fta = []
    ftpct = []
    
    urlstring = 'http://www.basketball-reference.com/leagues/NBA_'+str(season)+'_totals.html'

    source = urllib.request.urlopen(urlstring)
    source = source.read()
    soup = bs4.BeautifulSoup(source, 'lxml')
    totalstable = soup.find('table',attrs={'class':'sortable  stats_table'})
    datarows = totalstable.findAll('tr',attrs={'class':'full_table'})

    for i,row in enumerate(datarows):
        year.append(season)
        player.append(row.contents[3].text)
        playerid.append(row.contents[3].find('a')['href'].split('/')[-1].split('.')[0])
        pos.append(row.contents[5].text)
        age.append(row.contents[7].text)
        team.append(row.contents[9].text)
        ft.append(row.contents[37].text)
        fta.append(row.contents[39].text)
        ftpct.append(row.contents[41].text)

    data = {'year':year,'player':player,'playerid':playerid,'team':team,'pos':pos,'age':age,'ft':ft,'fta':fta,'ftpct':ftpct}

    oneYear = pd.DataFrame(data,columns=['year','player','playerid','team','pos','age','ft','fta','ftpct'])
    oneYear.to_sql('totals',engine,if_exists='append')

    pbar2.increment()
pbar2.finish()

### Adding Career-to-date

In [ ]:
query = "SELECT * FROM seasontotals"
allseasons = psql.read_sql(query,conn)
newseasons = allseasons[allseasons.year.astype(int) >= 2002].reset_index()

In [ ]:
ctd_fts = np.empty((3,newseasons.shape[0]))
for i in range(newseasons.shape[0]):
    rowsToCount = (allseasons.year.astype(int)<newseasons.ix[i].year.astype(int)).values & (allseasons.playerid==newseasons.ix[i].playerid).values
    ctd_fts[0,i]= allseasons[rowsToCount].fta.astype(int).sum()
    ctd_fts[1,i]=allseasons[rowsToCount].ft.astype(int).sum()
    ctd_fts[2,i] = rowsToCount.sum()

In [ ]:
ctd_columns = pd.DataFrame(ctd_fts.transpose(), columns=['ctd_fta','ctd_ft','years_in_league'])
ctd_columns['ctd_ftpct'] = ctd_columns.ctd_ft.astype(int)/ctd_columns.ctd_fta.astype(int)

newseasons = newseasons.join(ctd_columns)
newseasons = newseasons.drop('level_0',1)

In [ ]:
newseasons.to_sql('seasonwithcareer',engine,if_exists='append')

### Casting to Proper Data Types

In [ ]:
cur.execute("ALTER TABLE seasonwithcareer ALTER COLUMN fta TYPE bigint USING fta::bigint")
conn.commit()

In [ ]:
cur.execute("ALTER TABLE seasonwithcareer ALTER COLUMN ft TYPE bigint USING ft::bigint")
conn.commit()

In [ ]:
cur.execute("ALTER TABLE seasonwithcareer ALTER COLUMN age TYPE bigint USING age::bigint")
conn.commit()

In [ ]:
cur.execute("ALTER TABLE seasontotals ALTER COLUMN ft TYPE bigint USING ft::bigint")
conn.commit()

In [ ]:
cur.execute("ALTER TABLE seasontotals ALTER COLUMN fta TYPE bigint USING fta::bigint")
conn.commit()